In [1]:
%matplotlib inline
import mxnet as mx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import logging
import urllib
from PIL import Image
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

In [3]:
def get_model(prefix, epoch):
    download(prefix+'-symbol.json')
    download(prefix+'-%04d.params' % (epoch,))

# get_model('http://data.dmlc.ml/models/imagenet/resnet/152-layers/resnet-152', 0)
sym, arg_params, aux_params = mx.model.load_checkpoint('resnet-152', 0)

In [4]:
def get_fine_tune_model(symbol, arg_params, num_classes, layer_name='flatten0'):
    """
    symbol: the pretrained network symbol
    arg_params: the argument parameters of the pretrained model
    num_classes: the number of classes for the fine-tune datasets
    layer_name: the layer name before the last fully-connected layer
    """
    all_layers = symbol.get_internals()
    net = all_layers[layer_name+'_output']
    net = mx.symbol.FullyConnected(data=net, num_hidden=512, name='fc1')
    net = mx.symbol.FullyConnected(data=net, num_hidden=num_classes, name='fc2')
    net = mx.symbol.SoftmaxOutput(data=net, name='softmax')
    new_args = dict({k:arg_params[k] for k in arg_params if 'fc1' not in k})
    return (net, new_args)

In [5]:
batch_size = 32
# data_train = mx.image.ImageIter(path_imgrec='rectest.rec', batch_size=batch_size, data_shape=(3, 64, 64), label_width=120)
data_train = mx.image.ImageIter(path_imgrec='train.rec', batch_size=batch_size, data_shape=(3, 256, 256), label_width=120)
data_test = mx.image.ImageIter(path_imgrec='test.rec', batch_size=batch_size, data_shape=(3, 256, 256), label_width=120)
# lenet = get_net()

In [6]:
logging.getLogger().setLevel(logging.DEBUG)

def fit(symbol, arg_params, aux_params, train, val, batch_size, num_gpus):
    devs = [mx.gpu(i) for i in range(num_gpus)]
    mod = mx.mod.Module(symbol=symbol, context=devs)
    mod.fit(train, val,
        num_epoch=100,
        arg_params=arg_params,
        aux_params=aux_params,
        allow_missing=True,
        batch_end_callback = mx.callback.Speedometer(batch_size, 50),
        kvstore='device',
        optimizer='adam',
        optimizer_params={'learning_rate':0.00000001},
        initializer=mx.init.Xavier(rnd_type='gaussian', factor_type="in", magnitude=2),
        eval_metric='acc')
    metric = mx.metric.Accuracy()
    return mod.score(val, metric)

(new_sym, new_args) = get_fine_tune_model(sym, arg_params, 120)
mod_score = fit(new_sym, new_args, aux_params, data_train, data_test, batch_size, 2)

INFO:root:Epoch[0] Batch [50]	Speed: 113.63 samples/sec	accuracy=0.009804
INFO:root:Epoch[0] Batch [100]	Speed: 112.08 samples/sec	accuracy=0.008125
INFO:root:Epoch[0] Batch [150]	Speed: 111.08 samples/sec	accuracy=0.005000
INFO:root:Epoch[0] Batch [200]	Speed: 103.49 samples/sec	accuracy=0.006250
INFO:root:Epoch[0] Train-accuracy=0.010870
INFO:root:Epoch[0] Time cost=68.068
INFO:root:Epoch[0] Validation-accuracy=0.009465
INFO:root:Epoch[1] Batch [50]	Speed: 110.77 samples/sec	accuracy=0.009804
INFO:root:Epoch[1] Batch [100]	Speed: 103.61 samples/sec	accuracy=0.008125
INFO:root:Epoch[1] Batch [150]	Speed: 103.55 samples/sec	accuracy=0.005000
INFO:root:Epoch[1] Batch [200]	Speed: 103.66 samples/sec	accuracy=0.006250
INFO:root:Epoch[1] Train-accuracy=0.010870
INFO:root:Epoch[1] Time cost=68.257
INFO:root:Epoch[1] Validation-accuracy=0.009766
INFO:root:Epoch[2] Batch [50]	Speed: 108.82 samples/sec	accuracy=0.009804
INFO:root:Epoch[2] Batch [100]	Speed: 107.34 samples/sec	accuracy=0.008125

INFO:root:Epoch[19] Batch [100]	Speed: 104.29 samples/sec	accuracy=0.000000
INFO:root:Epoch[19] Batch [150]	Speed: 105.99 samples/sec	accuracy=0.000000
INFO:root:Epoch[19] Batch [200]	Speed: 105.59 samples/sec	accuracy=0.000000
INFO:root:Epoch[19] Train-accuracy=0.000000
INFO:root:Epoch[19] Time cost=67.626
INFO:root:Epoch[19] Validation-accuracy=0.000000
INFO:root:Epoch[20] Batch [50]	Speed: 108.78 samples/sec	accuracy=0.000000
INFO:root:Epoch[20] Batch [100]	Speed: 105.84 samples/sec	accuracy=0.000000
INFO:root:Epoch[20] Batch [150]	Speed: 106.05 samples/sec	accuracy=0.000000
INFO:root:Epoch[20] Batch [200]	Speed: 105.84 samples/sec	accuracy=0.000000
INFO:root:Epoch[20] Train-accuracy=0.000000
INFO:root:Epoch[20] Time cost=67.448
INFO:root:Epoch[20] Validation-accuracy=0.000000
INFO:root:Epoch[21] Batch [50]	Speed: 108.64 samples/sec	accuracy=0.000000
INFO:root:Epoch[21] Batch [100]	Speed: 104.48 samples/sec	accuracy=0.000000
INFO:root:Epoch[21] Batch [150]	Speed: 105.99 samples/sec	

INFO:root:Epoch[38] Batch [100]	Speed: 106.91 samples/sec	accuracy=0.000000
INFO:root:Epoch[38] Batch [150]	Speed: 106.56 samples/sec	accuracy=0.000000
INFO:root:Epoch[38] Batch [200]	Speed: 104.91 samples/sec	accuracy=0.000000
INFO:root:Epoch[38] Train-accuracy=0.000000
INFO:root:Epoch[38] Time cost=67.417
INFO:root:Epoch[38] Validation-accuracy=0.000000
INFO:root:Epoch[39] Batch [50]	Speed: 108.87 samples/sec	accuracy=0.000000
INFO:root:Epoch[39] Batch [100]	Speed: 106.91 samples/sec	accuracy=0.000000
INFO:root:Epoch[39] Batch [150]	Speed: 106.75 samples/sec	accuracy=0.000000
INFO:root:Epoch[39] Batch [200]	Speed: 105.97 samples/sec	accuracy=0.000000
INFO:root:Epoch[39] Train-accuracy=0.000000
INFO:root:Epoch[39] Time cost=67.145
INFO:root:Epoch[39] Validation-accuracy=0.000000
INFO:root:Epoch[40] Batch [50]	Speed: 109.34 samples/sec	accuracy=0.000000
INFO:root:Epoch[40] Batch [100]	Speed: 105.79 samples/sec	accuracy=0.000000
INFO:root:Epoch[40] Batch [150]	Speed: 105.76 samples/sec	

INFO:root:Epoch[57] Batch [100]	Speed: 105.78 samples/sec	accuracy=0.000000
INFO:root:Epoch[57] Batch [150]	Speed: 105.78 samples/sec	accuracy=0.000000
INFO:root:Epoch[57] Batch [200]	Speed: 105.79 samples/sec	accuracy=0.000000
INFO:root:Epoch[57] Train-accuracy=0.000000
INFO:root:Epoch[57] Time cost=67.478
INFO:root:Epoch[57] Validation-accuracy=0.000000
INFO:root:Epoch[58] Batch [50]	Speed: 108.39 samples/sec	accuracy=0.000000
INFO:root:Epoch[58] Batch [100]	Speed: 106.03 samples/sec	accuracy=0.000000
INFO:root:Epoch[58] Batch [150]	Speed: 106.01 samples/sec	accuracy=0.000000
INFO:root:Epoch[58] Batch [200]	Speed: 105.96 samples/sec	accuracy=0.000000
INFO:root:Epoch[58] Train-accuracy=0.000000
INFO:root:Epoch[58] Time cost=67.436
INFO:root:Epoch[58] Validation-accuracy=0.000000
INFO:root:Epoch[59] Batch [50]	Speed: 108.33 samples/sec	accuracy=0.000000
INFO:root:Epoch[59] Batch [100]	Speed: 105.56 samples/sec	accuracy=0.000000
INFO:root:Epoch[59] Batch [150]	Speed: 105.92 samples/sec	

INFO:root:Epoch[76] Batch [100]	Speed: 105.92 samples/sec	accuracy=0.000000
INFO:root:Epoch[76] Batch [150]	Speed: 106.20 samples/sec	accuracy=0.000000
INFO:root:Epoch[76] Batch [200]	Speed: 105.96 samples/sec	accuracy=0.000000
INFO:root:Epoch[76] Train-accuracy=0.000000
INFO:root:Epoch[76] Time cost=67.384
INFO:root:Epoch[76] Validation-accuracy=0.000000
INFO:root:Epoch[77] Batch [50]	Speed: 109.88 samples/sec	accuracy=0.000000
INFO:root:Epoch[77] Batch [100]	Speed: 106.41 samples/sec	accuracy=0.000000
INFO:root:Epoch[77] Batch [150]	Speed: 106.02 samples/sec	accuracy=0.000000
INFO:root:Epoch[77] Batch [200]	Speed: 105.92 samples/sec	accuracy=0.000000
INFO:root:Epoch[77] Train-accuracy=0.000000
INFO:root:Epoch[77] Time cost=67.153
INFO:root:Epoch[77] Validation-accuracy=0.000000
INFO:root:Epoch[78] Batch [50]	Speed: 108.95 samples/sec	accuracy=0.000000
INFO:root:Epoch[78] Batch [100]	Speed: 106.30 samples/sec	accuracy=0.000000
INFO:root:Epoch[78] Batch [150]	Speed: 106.30 samples/sec	

INFO:root:Epoch[95] Batch [100]	Speed: 106.49 samples/sec	accuracy=0.000000
INFO:root:Epoch[95] Batch [150]	Speed: 105.71 samples/sec	accuracy=0.000000
INFO:root:Epoch[95] Batch [200]	Speed: 105.80 samples/sec	accuracy=0.000000
INFO:root:Epoch[95] Train-accuracy=0.000000
INFO:root:Epoch[95] Time cost=67.374
INFO:root:Epoch[95] Validation-accuracy=0.000000
INFO:root:Epoch[96] Batch [50]	Speed: 108.81 samples/sec	accuracy=0.000000
INFO:root:Epoch[96] Batch [100]	Speed: 105.82 samples/sec	accuracy=0.000000
INFO:root:Epoch[96] Batch [150]	Speed: 105.63 samples/sec	accuracy=0.000000
INFO:root:Epoch[96] Batch [200]	Speed: 106.18 samples/sec	accuracy=0.000000
INFO:root:Epoch[96] Train-accuracy=0.000000
INFO:root:Epoch[96] Time cost=67.439
INFO:root:Epoch[96] Validation-accuracy=0.000000
INFO:root:Epoch[97] Batch [50]	Speed: 108.40 samples/sec	accuracy=0.000000
INFO:root:Epoch[97] Batch [100]	Speed: 106.12 samples/sec	accuracy=0.000000
INFO:root:Epoch[97] Batch [150]	Speed: 105.57 samples/sec	